In [ ]:
import sqlite3
import re

# Function to establish a database connection
def get_db_connection():
    return sqlite3.connect('banquet_management_system.db', timeout=5)

# Create tables based on the assumed relational schema
def create_tables(conn):
    with conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Users (
                user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                first_name TEXT NOT NULL,
                last_name TEXT NOT NULL,
                address TEXT NOT NULL,
                attendee_type TEXT NOT NULL,
                email TEXT UNIQUE NOT NULL,
                password TEXT NOT NULL,
                mobile_number TEXT NOT NULL,
                affiliated_organization TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Banquets (
                banquet_id INTEGER PRIMARY KEY AUTOINCREMENT,
                banquet_name TEXT NOT NULL,
                date_time TEXT NOT NULL,
                address TEXT NOT NULL,
                location TEXT NOT NULL,
                contact_first_name TEXT NOT NULL,
                contact_last_name TEXT NOT NULL,
                available TEXT CHECK (available IN ('Y','N','R')),
                quota INTEGER NOT NULL
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Meals (
                meal_id INTEGER PRIMARY KEY AUTOINCREMENT,
                dish_name TEXT UNIQUE NOT NULL,
                meal_type TEXT NOT NULL,
                price REAL NOT NULL,
                special_cuisine TEXT,
                banquet_id INTEGER,
                FOREIGN KEY (banquet_id) REFERENCES Banquets (banquet_id)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Registrations (
                registration_id INTEGER PRIMARY KEY AUTOINCREMENT,
                attendee_id INTEGER,
                banquet_id INTEGER,
                drink_choice TEXT,
                meal_choice TEXT,
                remarks TEXT,
                registration_time TEXT NOT NULL,
                seat_number INTEGER,
                FOREIGN KEY (attendee_id) REFERENCES Users (user_id),
                FOREIGN KEY (banquet_id) REFERENCES Banquets (banquet_id)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Feedback (
                feedback_id INTEGER PRIMARY KEY AUTOINCREMENT,
                attendee_id INTEGER,
                banquet_id INTEGER,
                rating INTEGER CHECK (rating BETWEEN 1 AND 5),
                comments TEXT,
                feedback_time TEXT NOT NULL,
                FOREIGN KEY (attendee_id) REFERENCES Users (user_id),
                FOREIGN KEY (banquet_id) REFERENCES Banquets (banquet_id)
            )
        ''')  # Corrected indentation here
    conn.commit()
    cursor.close()
        
# Establish a connection to the database
conn = get_db_connection()

# INSERT SAMPLE DATASET BELOW:


# Function for user signup
def signup(conn):
    print("=== Signup ===")
    first_name = input("Enter your first name: ")
    last_name = input("Enter your last name: ")
    address = input("Enter your address: ")
    attendee_type = input("Enter attendee type (e.g., staff, student): ")
    email = input("Enter your email: ")
    password = input("Enter your password: ")
    mobile_number = input("Enter your mobile number (8 digits): ")
    affiliated_organization = input("Enter your affiliated organization: ")

    # Validate inputs
    if not all([first_name, last_name, address, attendee_type, email, password, mobile_number]):
        print("All fields are required.")
        return

    if not re.match(r'^[A-Za-z]+$', first_name) or not re.match(r'^[A-Za-z]+$', last_name):
        print("First and last names must contain only letters.")
        return

    if not re.match(r'^\d{8}$', mobile_number):
        print("Mobile number must be exactly 8 digits.")
        return

    if "@" not in email:
        print("Email must contain '@'.")
        return

    cursor = conn.cursor()
    try:
        with conn:
            cursor.execute('''
                INSERT INTO Users (first_name, last_name, address, attendee_type, email, password, mobile_number, affiliated_organization)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (first_name, last_name, address, attendee_type, email, password, mobile_number, affiliated_organization))
        print("Signup successful!")
    except sqlite3.IntegrityError:
        print("Error: Email already exists. Please try logging in.")
    finally:
        cursor.close()

# Function for user login
def login(conn):
    print("=== Login ===")
    email = input("Enter your email: ")
    password = input("Enter your password: ")
    
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Users WHERE email = ? AND password = ?', (email, password))
    user = cursor.fetchone()
    cursor.close()
    
    if user:
        print("Login successful!")
        return user  # Return user information for further actions
    else:
        print("Incorrect email or password. Please try again.")
        return None

# Function for administrator login
def admin_login():
    print("=== Admin Login ===")
    email = input("Enter admin email (admin@gmail.com): ")
    password = input("Enter admin password (admin@123): ")
    
    if email == "admin@gmail.com" and password == "admin@123":
        print("Admin login successful!")
        return True
    else:
        print("Incorrect admin credentials.")
        return False

# Function for administrator to create a banquet
def create_banquet(conn):
    banquet_name = input("Enter Banquet Name: ")

    # Loop until valid date and time is provided
    while True:
        date_time = input("Enter Date and Time (YYYY-MM-DD HH:MM): ")
        if validate_datetime(date_time):
            break
        else:
            print("Invalid date and time format. Please try again.")

    address = input("Enter Address: ")
    location = input("Enter Location: ")
    contact_first_name = input("Enter Contact Staff First Name: ")
    contact_last_name = input("Enter Contact Staff Last Name: ")

    # Loop until valid availability input is provided
    while True:
        available = input("Is it available? (Y/N): ").strip().upper()
        if available in ('Y', 'N'):
            break
        else:
            print("Invalid input. Please enter 'Y' for Yes or 'N' for No.")

    # Loop until valid quota is provided
    while True:
        try:
            quota = int(input("Enter Quota: "))
            if quota > 0:  # Ensure that quota is a positive integer
                break
            else:
                print("Quota must be a positive integer. Please try again.")
        except ValueError:
            print("Invalid input. Quota must be an integer. Please try again.")

    cursor = conn.cursor()  # Create cursor for banquet creation
    try:
        with conn:
            cursor.execute('''
                INSERT INTO Banquets (banquet_name, date_time, address, location, contact_first_name, contact_last_name, available, quota)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (banquet_name, date_time, address, location, contact_first_name, contact_last_name, available, quota))

            banquet_id = cursor.lastrowid
            print("Banquet created successfully with BIN:", banquet_id)

            # Loop to add meals with specific prompts and check for unique dish names
            meal_prompts = [
                ("beef", "Enter the name of the beef dish: "),
                ("chicken", "Enter the name of the chicken dish: "),
                ("fish", "Enter the name of the fish dish: "),
                ("vegetarian", "Enter the name of the vegetarian dish: ")
            ]

            for meal_type, prompt in meal_prompts:
                while True:
                    dish_name = input(prompt)

                    # Check for unique dish name before inserting
                    cursor.execute('''
                        SELECT COUNT(*) FROM Meals WHERE dish_name = ? AND banquet_id = ?
                    ''', (dish_name, banquet_id))
                    count = cursor.fetchone()[0]

                    if count == 0:
                        break  # Unique dish name, break the loop
                    else:
                        print("Dish name must be unique. Please enter a different name.")

                while True:
                    try:
                        price = float(input("Price: "))
                        if price >= 0:  # Ensure price is non-negative
                            break
                        else:
                            print("Price must be a non-negative number. Please try again.")
                    except ValueError:
                        print("Invalid input. Price must be a number. Please try again.")

                # Insert the meal with the predefined meal_type
                cursor.execute('''
                    INSERT INTO Meals (dish_name, meal_type, price, special_cuisine, banquet_id)
                    VALUES (?, ?, ?, ?, ?)
                ''', (dish_name, meal_type, price, "", banquet_id))  # Assuming special_cuisine is optional

            conn.commit()  # Commit all changes
    except sqlite3.OperationalError as e:
        print(f"Database error: {e}")
    finally:
        cursor.close()  # Close cursor

# Helper function to validate datetime format
def validate_datetime(date_time_str):
    from datetime import datetime
    try:
        datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
        return True
    except ValueError:
        return False

# Function to view available banquets
def view_available_banquets(conn):
    print("=== Available Banquets ===")
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Banquets WHERE available = "Y"')
    banquets = cursor.fetchall()
    
    for banquet in banquets:
        print(f"BIN: {banquet[0]}, Name: {banquet[1]}, Date: {banquet[2]}, Location: {banquet[4]}, Quota: {banquet[8]}")
        
    cursor.close()
    return banquets

# Function for report generation for a single banquet
def report_generation(conn):
    cursor = conn.cursor()

    # Prompt for the banquet ID (BIN)
    banquet_id = int(input("Enter the Banquet ID (BIN) to generate the report: "))

    # Get the most popular meal(s) for the specified banquet
    cursor.execute('''
        SELECT meal_choice, COUNT(*) AS count
        FROM Registrations r
        WHERE r.banquet_id = ?
        GROUP BY meal_choice
        ORDER BY count DESC
    ''', (banquet_id,))
    popular_meals = cursor.fetchall()

    if popular_meals:
        max_count = popular_meals[0][1]
        most_popular_meals = [meal[0] for meal in popular_meals if meal[1] == max_count]
        print(f"Most Popular Meal(s) for Banquet ID {banquet_id}: {', '.join(most_popular_meals)} with {max_count} registrations.")
    else:
        print(f"No meal registrations found for Banquet ID {banquet_id}.")

    # Calculate attendance rate for the specified banquet
    cursor.execute('''
        SELECT COUNT(*) AS attendance, b.quota
        FROM Registrations r
        JOIN Banquets b ON r.banquet_id = b.banquet_id
        WHERE r.banquet_id = ?
    ''', (banquet_id,))
    attendance_data = cursor.fetchone()

    if attendance_data:
        attendance, quota = attendance_data

        # Check if quota is None or zero
        if quota is not None and quota > 0:
            attendance_rate = (attendance / quota) * 100
        else:
            attendance_rate = 0  # Assign 0% if quota is None or 0

        print(f"\nBanquet ID: {banquet_id}, Attendance: {attendance}, Quota: {quota}, Attendance Rate: {attendance_rate:.2f}%")
    else:
        print(f"No data found for Banquet ID {banquet_id}.")

    # Get the average rating for the specified banquet
    cursor.execute('''
        SELECT AVG(rating) AS average_rating
        FROM Feedback
        WHERE banquet_id = ?
    ''', (banquet_id,))
    average_rating_data = cursor.fetchone()

    if average_rating_data and average_rating_data[0] is not None:
        average_rating = average_rating_data[0]
    else:
        average_rating = ''  # Assign empty string if there are no ratings

    print(f"Average Rating for Banquet ID {banquet_id}: {average_rating}")
    
    cursor.close()

# Function to search for banquets based on time or quota
def search_banquets(conn):
    print("=== Search Banquets ===")
    search_type = input("Search by time or quota? (time/quota): ").strip().lower()
    
    cursor = conn.cursor()
    
    if search_type == 'time':
        search_time = input("Enter the date and time (YYYY-MM-DD HH:MM): ")
        cursor.execute('SELECT * FROM Banquets WHERE date_time = ? AND available = "R"', (search_time,))
        results = cursor.fetchall()
        
        if results:
            print("=== Available Banquets Matching Time ===")
            for banquet in results:
                print(f"BIN: {banquet[0]}, Name: {banquet[1]}, Date: {banquet[2]}, Location: {banquet[4]}, Quota: {banquet[8]}")
        else:
            print("No available banquets found for the specified time.")
    
    elif search_type == 'quota':
        min_quota = int(input("Enter minimum quota: "))
        cursor.execute('SELECT * FROM Banquets WHERE quota >= ? AND available = "R"', (min_quota,))
        results = cursor.fetchall()
        
        if results:
            print("=== Available Banquets Matching Quota ===")
            for banquet in results:
                print(f"BIN: {banquet[0]}, Name: {banquet[1]}, Date: {banquet[2]}, Location: {banquet[4]}, Quota: {banquet[8]}")
        else:
            print("No available banquets found with the specified minimum quota.")
    
    else:
        print("Invalid search type. Please enter 'time' or 'quota'.")
    
    cursor.close()

# Function for banquet registration
def register_for_banquet(conn, attendee_id):
    cursor = conn.cursor()

    # Prompt for banquet ID (BIN)
    banquet_id = int(input("Enter the Banquet ID (BIN) you want to register for: "))

    # Check if the banquet exists and is available
    cursor.execute('SELECT available FROM Banquets WHERE banquet_id = ?', (banquet_id,))
    banquet = cursor.fetchone()

    if banquet is None:
        print("Banquet ID not found.")
        cursor.close()
        return

    if banquet[0] != 'Y':
        print("This banquet is not available for registration.")
        cursor.close()
        return

    # Check if the attendee is already registered for the specified banquet
    cursor.execute('''SELECT COUNT(*) FROM Registrations 
                      WHERE attendee_id = ? AND banquet_id = ?''', (attendee_id, banquet_id))
    registration_count = cursor.fetchone()[0]

    if registration_count > 0:
        print("You are already registered for this banquet.")
        cursor.close()
        return

    # Display available meal choices for the selected banquet
    cursor.execute('SELECT meal_id, dish_name, price FROM Meals WHERE banquet_id = ?', (banquet_id,))
    meals = cursor.fetchall()

    if not meals:
        print("No meals available for this banquet.")
        cursor.close()
        return

    print("Available meal choices:")
    for meal in meals:
        print(f"Meal ID: {meal[0]}, Dish Name: {meal[1]}, Price: ${meal[2]:.2f}")

    # Prompt for meal ID
    meal_id = int(input("Enter the Meal ID you want to choose: "))

    # Check if the meal ID is valid
    cursor.execute('SELECT * FROM Meals WHERE meal_id = ? AND banquet_id = ?', (meal_id, banquet_id))
    meal = cursor.fetchone()

    if meal is None:
        print("Invalid Meal ID or the meal does not belong to the specified banquet.")
        cursor.close()
        return

    # Collect registration details
    drink_choice = input("Enter your drink choice: ")
    remarks = input("Any remarks? ")
    registration_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Insert the registration into the database
    cursor.execute('''INSERT INTO Registrations 
                      (attendee_id, banquet_id, drink_choice, meal_choice, remarks, registration_time)
                      VALUES (?, ?, ?, ?, ?, ?)''', (attendee_id, banquet_id, drink_choice, meal_id, remarks, registration_time))

    conn.commit()  # Commit the registration
    print("Registration successful!")

    cursor.close()

# To import datetime at the top of the file
from datetime import datetime

# Function to update attendee profile
def update_profile(conn, attendee_id):
    print("=== Update Profile ===")
    cursor = conn.cursor()
    
    # Fetch current profile information
    cursor.execute('SELECT * FROM Users WHERE user_id = ?', (attendee_id,))
    user = cursor.fetchone()
    
    if user:
        print("Current Profile Information:")
        print(f"Address: {user[3]}")
        print(f"Attendee Type: {user[4]}")
        print(f"Mobile Number: {user[7]}")
        print(f"Affiliated Organization: {user[8]}")
        
        # Prompt for updates
        address = input("Enter new address (or press Enter to keep current): ")
        attendee_type = input("Enter new attendee type (or press Enter to keep current): ")
        mobile_number = input("Enter new mobile number (8 digits, or press Enter to keep current): ")
        affiliated_organization = input("Enter new affiliated organization (or press Enter to keep current): ")

        # Create a list to hold updated values
        updates = []
        query = 'UPDATE Users SET '
        
        if address:
            updates.append(f'address = "{address}"')
        if attendee_type:
            updates.append(f'attendee_type = "{attendee_type}"')
        if mobile_number and re.match(r'^\d{8}$', mobile_number):
            updates.append(f'mobile_number = "{mobile_number}"')
        if affiliated_organization:
            updates.append(f'affiliated_organization = "{affiliated_organization}"')

        # Only update if there are changes
        if updates:
            query += ', '.join(updates) + ' WHERE user_id = ?'
            cursor.execute(query, (attendee_id,))
            conn.commit()
            print("Profile updated successfully!")
        else:
            print("No updates made.")

    else:
        print("User not found.")
    
    cursor.close()
    
# Function for attendees to submit feedback for a banquet
def submit_feedback(conn, attendee_id):
    cursor = conn.cursor()
    
    # Prompt for banquet ID (BIN)
    banquet_id = int(input("Enter the Banquet ID (BIN) for your feedback: "))
    
    # Check if the attendee is registered for the specified banquet
    cursor.execute('''
        SELECT COUNT(*) FROM Registrations 
        WHERE attendee_id = ? AND banquet_id = ?
    ''', (attendee_id, banquet_id))
    
    registration_count = cursor.fetchone()[0]
    
    if registration_count == 0:
        print("You are not registered for this banquet. Feedback cannot be submitted.")
        cursor.close()
        return
    
    # Check if feedback already exists for this banquet
    cursor.execute('''
        SELECT COUNT(*) FROM Feedback 
        WHERE attendee_id = ? AND banquet_id = ?
    ''', (attendee_id, banquet_id))
    
    feedback_count = cursor.fetchone()[0]
    
    if feedback_count > 0:
        print("You have already submitted feedback for this banquet.")
        cursor.close()
        return
    
    # Get feedback details
    rating = int(input("Enter your rating (1-5): "))
    comments = input("Enter your comments: ")
    
    # Insert feedback into the database
    cursor.execute('''
        INSERT INTO Feedback (attendee_id, banquet_id, rating, comments, feedback_time)
        VALUES (?, ?, ?, ?, datetime('now'))
    ''', (attendee_id, banquet_id, rating, comments))
    
    conn.commit()
    cursor.close()
    print("Thank you for your feedback!")
    
# Function to retrieve attendee information and their registered banquets with feedback
def get_attendee_info(conn):
    email = input("Enter attendee email to retrieve information: ")
    cursor = conn.cursor()
    
    # Fetch user information
    cursor.execute('SELECT * FROM Users WHERE email = ?', (email,))
    user = cursor.fetchone()
    
    if user:
        print("Attendee Information:")
        print("First Name:", user[1])
        print("Last Name:", user[2])
        print("Address:", user[3])
        print("Attendee Type:", user[4])
        print("Email:", user[5])
        print("Mobile Number:", user[7])
        print("Affiliated Organization:", user[8])

        # Retrieve registered banquets for the attendee
        cursor.execute('''
            SELECT b.banquet_id, b.banquet_name, b.date_time 
            FROM Registrations r 
            JOIN Banquets b ON r.banquet_id = b.banquet_id 
            WHERE r.attendee_id = ?
        ''', (user[0],))
        
        registered_banquets = cursor.fetchall()
        
        if registered_banquets:
            print("\nRegistered Banquets:")
            for banquet in registered_banquets:
                print(f"Banquet ID: {banquet[0]}, Name: {banquet[1]}, Date: {banquet[2]}")
                
                # Retrieve feedback for the specific banquet
                cursor.execute('''
                    SELECT rating, comments, feedback_time 
                    FROM Feedback 
                    WHERE attendee_id = ? AND banquet_id = ?
                ''', (user[0], banquet[0]))
                
                feedback = cursor.fetchone()
                
                if feedback:
                    print(f"Feedback: Rating: {feedback[0]}, Comments: {feedback[1]}, Date: {feedback[2]}")
                else:
                    print("No feedback submitted for this banquet.")
        else:
            print("No registered banquets found for this attendee.")
    else:
        print("User not found.")
    
    cursor.close()
    
# Main menu for user login/signup
def main_menu(conn):
    while True:
        user_type = input("Are you an attendee or an administrator? (attendee/admin): ").strip().lower()
        if user_type == 'attendee':
            action = input("Do you want to login or signup? (login/signup): ").strip().lower()
            if action == 'signup':
                signup(conn)
            elif action == 'login':
                user = login(conn)
                if user:  # Only proceed if the login was successful
                    attendee_id = user[0]
                    print("Welcome, ", user[1])  # Display welcome message
                    while True:  # Enter attendee menu loop
                        print("\nAttendee Menu")
                        print("1. View Available Banquets")
                        print("2. Register for a Banquet")
                        print("3. Search for Registered Banquets")
                        print("4. Update Profile")
                        print("5. Submit Feedback")
                        print("6. Exit")
                        choice = input("Choose an option: ")

                        if choice == '1':
                            view_available_banquets(conn)
                        elif choice == '2':
                            current_attendee_id = user[0]
                            register_for_banquet(conn, current_attendee_id)
                        elif choice == '3':
                            search_banquets(conn)
                        elif choice == '4':
                            update_profile(conn, attendee_id)
                        elif choice == '5':
                            submit_feedback(conn, attendee_id)
                        elif choice == '6':
                            print("Exiting the system.")
                            return
                        else:
                            print("Invalid choice. Please try again.")
            else:
                print("Invalid option. Please enter 'login' or 'signup'.")
        elif user_type == 'admin':
            if admin_login():
                while True:
                    print("\nAdmin Menu")
                    print("1. Create Banquet")
                    print("2. Get Attendee Information")
                    print ("3. Generate a Report")
                    print("4. Exit")
                    choice = input("Choose an option: ")

                    if choice == '1':
                        create_banquet(conn)
                    elif choice == '2':
                        get_attendee_info(conn)
                    elif choice == '3':
                        report_generation(conn)
                    elif choice == '4':
                        print("Exiting the system.")
                        return
                    else:
                        print("Invalid choice. Please try again.")
        else:
            print("Invalid user type. Please enter 'attendee' or 'admin'.")

# Create tables and run the main menu
if __name__ == "__main__":
    conn = get_db_connection()
    create_tables(conn)
    main_menu(conn)

In [ ]:
# Use this to call the BMS after the first iteration when importing the testing dataset
# This is done to prevent the uniqueness constraint

main_menu(conn)